In [4]:
from autogen_agentchat.agents import AssistantAgent, SocietyOfMindAgent, UserProxyAgent
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_ext.models.openai import OpenAIChatCompletionClient
import asyncio

In [5]:
import os
from dotenv import load_dotenv

In [6]:
open_ai_key = os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
model = OpenAIChatCompletionClient(model="gpt-4o", api_key=open_ai_key)

In [7]:
from autogen_core.memory import MemoryContent, MemoryMimeType
from autogen_ext.memory.mem0 import Mem0Memory

In [8]:
from mem0 import MemoryClient
import json

In [9]:
with open("developer_profiles.json","r") as file:
    developer_files = json.load(file)

In [10]:
profiles = [
    "Alice is a developer with 6 years of experience. Skills: UI. Contact: alice@company.com. Current project load: 2. Works only on priority projects.",
    "Bob is a developer with 6 years of experience. Skills: UI. Contact: bob@company.com. Current project load: 1. Available for any projects.",
    "Carol is a developer with 8 years of experience. Skills: Backend. Contact: carol@company.com. Current project load: 4. Available for any projects.",
    "Dan is a developer with 5 years of experience. Skills: Backend. Contact: dan@company.com. Current project load: 2. Available for any projects.",
    "Eve is a developer with 5 years of experience. Skills: Security. Contact: eve@company.com. Current project load: 3. Works only on priority projects.",
    "Frank is a developer with 4 years of experience. Skills: Mobile. Contact: frank@company.com. Current project load: 3. Available for any projects.",
    "Grace is a developer with 10 years of experience. Skills: Billing. Contact: grace@company.com. Current project load: 1. Available for any projects.",
    "Heidi is a developer with 3 years of experience. Skills: Billing. Contact: heidi@company.com. Current project load: 4. Works only on priority projects.",
    "Ivan is a developer with 4 years of experience. Skills: Localization. Contact: ivan@company.com. Current project load: 1. Works only on priority projects.",
    "Judy is a developer with 9 years of experience. Skills: Analytics. Contact: judy@company.com. Current project load: 2. Available for any projects.",
    "Mallory is a developer with 3 years of experience. Skills: Auth. Contact: mallory@company.com. Current project load: 3. Works only on priority projects.",
    "Trent is a developer with 4 years of experience. Skills: Dashboard. Contact: trent@company.com. Current project load: 3. Available for any projects.",
    "Victor is a developer with 10 years of experience. Skills: Search. Contact: victor@company.com. Current project load: 5. Available for any projects."
]

In [11]:
load_dotenv()
meme0_api = os.getenv("MEM0")
#client = MemoryClient(api_key = meme0_api)

In [12]:
client = Mem0Memory(
    user_id="developers",
    api_key=meme0_api,
    is_cloud=True,)

In [13]:
for profile in profiles:
    client._client.add(messages=[
        {
            'role':'user','content': profile
        },
        
    ], user_id='developer_profiles'
                       )




'''client._client.add(messages=[
            {
                "role":'user',"content": "I am a vegetarian food and I love cooking.",
            },
            {
                "role":'assistant',"content": "I will remember that preference is Vegetarian.",
            },
        ],user_id='developer_profile')'''

c:\Users\Chinmay\anaconda3\envs\'SoM_project'\Lib\site-packages\mem0\client\utils.py:18: DeprecationWarning: output_format='v1.0' is deprecated therefore setting it to 'v1.1' by default. Check out the docs for more information: https://docs.mem0.ai/platform/quickstart#4-1-create-memories
  return func(*args, **kwargs)


'client._client.add(messages=[\n            {\n                "role":\'user\',"content": "I am a vegetarian food and I love cooking.",\n            },\n            {\n                "role":\'assistant\',"content": "I will remember that preference is Vegetarian.",\n            },\n        ],user_id=\'developer_profile\')'

In [14]:
bug_queue_mgr_agent = AssistantAgent(
    name = "bug_queue_manager_agent",
    description = "Manages the queue of bug reports and distributes them to inner teams",
    model_client = model,
    system_message = """
        You will receive the bugs from user in the below example format:
        {
            "id": "BR-0001",
            "title": "Unexpected logout after file upload",
            "description": "When trying to onboarding, the system responds with an error or no response at all.",
            "reported_by": "Customer_Support",
            "severity_hint": "Medium",
            "component": "Security",
            "timestamp": "2025-08-02T21:53:00Z",
            "labels": [
            "login"
            ],
        }
        your task is to distribute these bugs to the triage_agents for further assignment based on their current workload.
        you will output the format in the same way as you receive it, just adding the name of the triage_agent in the bug json format, so that 
        it will be selected for further processing of the bug.
                    """
)

In [15]:
review_aggregator_agent = AssistantAgent(
    name = "review_agent",
    description = "Collects inner team outputs and prepares a triage report for human review",
    model_client = model,
    system_message = """
        You will receive the output from the triage team which will have all the bug details along with the updates 
        from triage team, your task is to create a triage report and give it to the human review
        """
)

In [16]:
outer_human_agent = UserProxyAgent(
    name = "outer_proxy_agent",
    description="You are a proxy agent",
    input_func = input,)

In [17]:
bug_classifier_agent = AssistantAgent(
    name = "bug_classifier_agent",
    description= "This agent classifies the bug",
    model_client = model,
    system_message = """
                    You are an experienced bug classifier having expertise in accurately understanding the description of 
                    the given bug which is provided by the key 'description'. After carefull assessment of the bug description,
                    you will classify the bug into the correct class and update the component key of the bug format. 
                    Once your task is complete, you will output exactly the updates values and keys of the bug format
                        """
)

In [18]:
priority_assessor_agent = AssistantAgent(
    name = "priority_assessor_agent",
    description = "This agent assess the priority of the bug",
    model_client = model,
    system_message = """
                    You are an experienced bug priority assessor having an expertise in understanding all the details of the
                    bug format very properly and understand the priority of the given bug. Once you assess the priority, 
                    you have to update the 'severity_hint' key of the bug format and then output the updated bug format."""
)

In [20]:
assignment_agent = AssistantAgent(
    name = "assignment_agent",
    description = "This agent assigns the bug to the correct developer",
    model_client = model,
    memory = [client],
    system_message = """
                    You are an experienced assigment agent having an expertise in understanding all the details of the bug
                    format very properly, accounting the load on the developers, you will assign the bugs to the available and
                    correct developers appropriately. Once you are done with your task, you will update the bug format with the
                    new assignment key having value with the developer's name."""
)

In [21]:
inner_human_agent = UserProxyAgent(
    name = "inner_agent",
    description = "You are a proxy agent",
    input_func = input
)

In [22]:
triage_team = RoundRobinGroupChat(
    name = "triage_team",
    description = "This team perform the bug analysis and updates the bug format with updates priority, ",
    participants = [bug_classifier_agent, priority_assessor_agent, assignment_agent, inner_human_agent],
    max_turns = 8,
    termination_condition = TextMentionTermination(text="Approve")
)

In [23]:
incident_management_team = SelectorGroupChat(
    name = "incident_management_team",
    allow_repeated_speaker = True,
    description = "This team triages the bugs and then writes the incident report",
    participants = [bug_queue_mgr_agent, triage_team, review_aggregator_agent, outer_human_agent],
    max_turns = 8,
    model_client= model
)

In [32]:
task = """
    {
    "id": "BR-0003",
    "title": "Search not returning results for empty strings",
    "description": "Incorrect results returned when querying for empty strings.",
    "reported_by": "Product_Manager",
    "severity_hint": "Critical",
    "component": "Dashboard",
    "timestamp": "2025-08-02T16:43:00Z",
    "labels": [
      "error",
      "checkout",
      "login"
    ],}

"""

In [27]:
from autogen_agentchat.ui import Console

async def run(task):
    await Console(incident_management_team.run_stream(task=task))

In [28]:
await run(task=task)

---------- TextMessage (user) ----------

    {
    "id": "BR-0001",
    "title": "Unexpected logout after file upload",
    "description": "When trying to onboarding, the system responds with an error or no response at all.",
    "reported_by": "Customer_Support",
    "severity_hint": "Medium",
    "component": "Security",
    "timestamp": "2025-08-02T21:53:00Z",
    "labels": [
      "login"
    ],


---------- TextMessage (bug_queue_manager_agent) ----------
```json
{
    "id": "BR-0001",
    "title": "Unexpected logout after file upload",
    "description": "When trying to onboarding, the system responds with an error or no response at all.",
    "reported_by": "Customer_Support",
    "severity_hint": "Medium",
    "component": "Security",
    "timestamp": "2025-08-02T21:53:00Z",
    "labels": [
        "login"
    ],
    "triage_agent": "Agent_Smith"
}
```
---------- TextMessage (bug_classifier_agent) ----------
```json
{
    "id": "BR-0001",
    "title": "Unexpected logout after 

In [31]:
await triage_team.reset()
await incident_management_team.reset()